# Session 1: homework

For the homework, we found a great dataset to practice your linear regression skills on. This dataset has been used in the following publication:

> P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. 'Modeling wine preferences by data mining from physicochemical properties.' *Decision Support Systems* 47(4):547-553.

Don't copypaste the code for utils.py, but call a fucntion that you need (e.g. utils.train())


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
# %cd gdrive/MyDrive/ML2022/session-1
%cd gdrive/MyDrive/my_ml_2023/session-6

Mounted at /content/gdrive
/content/gdrive/MyDrive/my_ml_2023/session-6


In [ ]:
import utils
import numpy as np
import torch
from torch import nn


# to get reproducible results:
torch.manual_seed(1234)
np.random.seed(1234)



## Preprocessing

Load the dataset (it's also included in the session's folder):

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv

--2023-11-09 16:33:49--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘winequality-white.csv’

winequality-white.c     [ <=>                ] 258.23K  --.-KB/s    in 0.04s   

2023-11-09 16:33:49 (5.76 MB/s) - ‘winequality-white.csv’ saved [264426]



You can find the dataset (encoded as csv-files) and load it using `pandas`:

In [ ]:
import pandas as pd
white = pd.read_csv('winequality-white.csv', sep=';')

Let us inspect 10 random instances from the dataset:

In [ ]:
white.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1431,6.1,0.22,0.49,1.5,0.051,18.0,87.0,0.99280,3.30,0.46,9.6,5
445,7.1,0.32,0.32,11.0,0.038,16.0,66.0,0.99370,3.24,0.40,11.5,3
2816,7.2,0.17,0.41,1.6,0.052,24.0,126.0,0.99228,3.19,0.49,10.8,5
4049,6.8,0.16,0.36,1.3,0.034,32.0,98.0,0.99058,3.02,0.58,11.3,6
4779,6.0,0.59,0.00,0.8,0.037,30.0,95.0,0.99032,3.10,0.40,10.9,4
142,7.9,0.21,0.40,1.2,0.039,38.0,107.0,0.99200,3.21,0.54,10.8,6
2703,6.5,0.23,0.36,16.3,0.038,43.0,133.0,0.99924,3.26,0.41,8.8,5
3252,7.1,0.26,0.37,5.5,0.025,31.0,105.0,0.99082,3.06,0.33,12.6,8
4282,5.7,0.26,0.24,17.8,0.059,23.0,124.0,0.99773,3.30,0.50,10.1,5
46,6.2,0.45,0.26,4.4,0.063,63.0,206.0,0.99400,3.27,0.52,9.8,4


This dataset records, for a large number of (Portugese) wines, a number of objective "physiochemical" properties, such as various kinds of acidity or the level of chlorides. In the very last column ("quality"), it contains a subjective rating for that wine, as an integer  score, ranging from 1 ("very bad") to 10 ("excellent"). This score is a median rating given by at least 3 evaluations made by wine experts.

We'll extract the "quality" column as our $y$ (target value):

In [ ]:
y_label = white['quality'].values
type(y_label)
y_label[:30]

array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 7, 5, 7, 6, 8, 6, 5, 8, 7,
       8, 5, 6, 6, 6, 6, 6, 7])

We can then drop that column from the dataframe (so that all that remains are input features)

In [ ]:
white = white.drop(['quality'], axis=1)
white.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
3474,6.5,0.22,0.19,1.10,0.064,36.0,191.0,0.99297,3.05,0.50,9.5
3337,6.9,0.38,0.29,13.65,0.048,52.0,189.0,0.99784,3.00,0.60,9.5
1165,6.0,0.24,0.32,6.30,0.030,34.0,129.0,0.99460,3.52,0.41,10.4
1281,7.7,0.27,0.34,1.80,0.028,26.0,168.0,0.99110,2.99,0.48,12.1
2377,7.0,0.22,0.29,8.90,0.050,24.0,90.0,0.99556,3.29,0.46,9.8
4392,6.2,0.25,0.44,15.80,0.057,39.0,167.0,0.99804,3.14,0.51,9.2
840,7.6,0.33,0.35,6.30,0.036,12.0,126.0,0.99240,3.16,0.39,12.0
1755,6.3,0.25,0.23,14.90,0.039,47.0,142.0,0.99705,3.14,0.35,9.7
3483,6.1,0.36,0.26,8.15,0.035,14.0,88.0,0.99031,3.06,0.27,13.0
2714,6.8,0.24,0.37,7.45,0.043,59.0,188.0,0.99579,3.20,0.50,9.4


To extract these features as a purely numeric numpy array, we can do:

In [ ]:
X = white.values

Print the type and shape of X:

In [ ]:
X.shape
type(X)

numpy.ndarray

Let us divide the available data into a train set and test set. It's important to **stratify** these splits in terms of ratings, in order to make sure that we have a similar distribution of ratings in train and test. Scikit-learn has a very convenient function for that:

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_dev_test, y_train, y_dev_test = train_test_split(X, y_label, test_size=0.4,
                                                    random_state=1234, stratify=y_label)

x_dev, x_test, y_dev, y_test = train_test_split(x_dev_test, y_dev_test, test_size=0.5,
                                                    random_state=1234, stratify=y_dev_test)

Note that the stratify argument is crucial. Verify the shapes:

In [ ]:
print(x_train.shape)
print(x_dev.shape)
print(x_test.shape)
print(y_train.shape)
print(y_dev.shape)
print(y_test.shape)

(2938, 11)
(980, 11)
(980, 11)
(2938,)
(980,)
(980,)


0. There's one final bit we need to take care of and that is the normalization of our data. Inspect the feature values in the dataframe above usinf `describe()`, you'll notice that the features cover very different ranges. To account for that it's best to normalize our data. Again: `sklearn` to the rescue! Use `StandardScaler` to normalize the data.

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000


In [ ]:
from sklearn.preprocessing import StandardScaler

# conventional standardization: remove mean and divide by std
scaler =
x_train =
x_dev =
x_test =


y_scaler =

# we add a new dimension to y's (it had just one) to fulfil requirements of StandardScaler (2 dimensions)

y_train = np.expand_dims(y_train, axis=1)
y_dev = np.expand_dims(y_dev, axis=1)

y_train =
y_dev =

(2938, 11)
(980, 11)


Two things to note here :
- We only use `x_train` to determine the parameters of our scalar (i.e. the mean and std): cf. `.fit_transform()`
- We apply the fitted scaler to `x_test`, so that we really pretend that we haven't seen the test data yet

Obviously, the shape of our matrices isn't affected.

The following fucntion will map our arrays into data loaders:

In [ ]:
def np2set(x, y, shuffle):
  x = torch.tensor(x, dtype=torch.float)
  y =  torch.tensor(y, dtype=torch.float)

  dataset = torch.utils.data.TensorDataset(x, y)
  iterator = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=shuffle)
  return iterator


## Linear regression

1. Produce an exploratory scatterplot for the training data with `regplot()`. Explore whether you could potentially predict a wine's appreciation level (y_train) by the percentage of alcohol in it (the last feature column in `x_train`). Add proper labels to your x- and y-axes. Describe what you see.

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
# code here


2. Base yourself on the code from the notebook in class (call functions from utils.py, but do not copypaste them) and implement a **univariate linear regression** that aims to predict a wine appreciation from the "alcohol" column. Train this model. Use the SGD-optimizer with a learning rate of "0.1" and train for 10 epochs.

In [ ]:
u_train_iter =  np2set(x_train[:, [10]], y_train, shuffle=True)
u_dev_iter =  np2set(x_dev[:, [10]], y_dev, shuffle=False)
u_test_iter = np2set(x_test[:, [10]], y_test, shuffle=False)

3. Re-run the cell above, but this time, use a learning rate of "0.0001". What changes in the output? Is that good or bad?

4. Retrain the network with a learning rate of 0.001, but this time train for 40 epochs. At which epoch does the loss become stable? Describe in your own words what that means. Plot the loss.

5. Retrain the network with a learning rate of 0.001, but this time train for 35 epochs.

6. Predict the appreciations for the wines in the test data. Note that we consider the last column of `x_test`. Calculate the **mean squared error** and **mean absolute error**  between the predictions and the ground truth (in `y_test`). Also: assess this result in your own words: what does this MSE imply in interpretive terms? Plot the training data samples in a scatterplot and add the regression line which the network learned.

In [2]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae




7. Print correlations draw pair plots (`pairplot`) between the target variables and features. What do you observe? Next on, fit a multivariate network to use all of the feature columns in `x_train`. Adapt the network architecture as necessary (in particular, you'll want to change the number of input features). Use a learning rate of "0.001" and train for 35 epochs. Compare the losses being printed to the univariate model that we ran above and describe what you see.

8. Predict the appreciation ratings for the test data (i.e. the full `x_test` this time) and calculate the MSE and MAE. Did the test results improve? Why (not), you think?

9. Go through all of the preprocessing steps above again, but this time for the red wine dataset (loading, cleaning, splitting, scaling, etc.). Make sure to copy all the relevant code, so that in the code blocks below, you again obtain a suitable `x_train`, `x_test`, `y_train`, and `y_test` but this time for the *red* wines. You can obtain the relevant red wine dataset as follows:

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv

--2023-11-09 16:42:14--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘winequality-red.csv’

winequality-red.csv     [ <=>                ]  82.23K  --.-KB/s    in 0.03s   

2023-11-09 16:42:14 (3.08 MB/s) - ‘winequality-red.csv’ saved [84199]



In [ ]:
red = pd.read_csv('winequality-red.csv', sep=';')
y = red['quality'].values
red = red.drop(['quality'], axis=1)
X = red.values

(320, 11)


10. Train a univariate network to model the effect of alcoholic percentage on the *red* wines' appreciations. (You can determine a suitable learning rate and number of epochs yourself...) Print the parameters learned by your model: is the effect similar to that for the white wines? Plot the regression line.

12. Fit a multivariate network to the red wines dataset, this time using *all* of the available feature columns in `X_train`. You can play around with the learning rate and the number of epochs to optimize your performance on the training data. Compute the performance of your trained model on the test data, using the mean squared error. Compare the MSE and MAE to the univariate case.